# SpikingJelly Linear 层实验分析

实验验证：Linear 层的输入/输出是什么？W 扮演什么角色？Linear + Neuron 组合后行为如何？

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from spikingjelly.activation_based import neuron, layer, surrogate

matplotlib.rcParams['font.sans-serif'] = ['Noto Sans CJK JP', 'Droid Sans Fallback', 'DejaVu Sans']
matplotlib.rcParams['axes.unicode_minus'] = False

torch.manual_seed(42)
print('环境加载成功')

## 1. 单独的 Linear 层：输入/输出类型

In [ ]:
# 创建一个 Linear 层
linear = layer.Linear(4, 3, bias=False, step_mode='s')

# 手动设置权重，方便观察
with torch.no_grad():
    linear.weight.copy_(torch.tensor([
        [1.0, 0.5, -0.3, 0.2],
        [0.1, 0.8, 0.4, -0.5],
        [-0.2, 0.3, 0.9, 0.1]
    ]))

print('W (权重矩阵):')
print(linear.weight.data)
print(f'\nW 形状: {linear.weight.shape} = [out_features={3}, in_features={4}]')

In [ ]:
# 测试1：实数输入
x_real = torch.tensor([0.5, 1.2, -0.3, 0.8])
y_real = linear(x_real)

print('=== 实数输入 ===')
print(f'输入: {x_real.tolist()} (实数)')
print(f'输出: {y_real.tolist()} (实数)')
print(f'手动验证 W @ x: {(linear.weight.data @ x_real).tolist()}')
print(f'输出 == W @ x: {torch.allclose(y_real, linear.weight.data @ x_real)}')

In [ ]:
# 测试2：二值(spike)输入
x_spike = torch.tensor([1.0, 0.0, 1.0, 0.0])  # 模拟spike: 神经元0和2发放
y_spike = linear(x_spike)

print('=== Spike 输入 (0/1) ===')
print(f'输入: {x_spike.tolist()} (二值: 神经元0和2发放)')
print(f'输出: {y_spike.tolist()} (实数)')
print(f'\n物理含义: 输出 = W的第0列 + W的第2列（发放神经元对应的列之和）')
print(f'W[:, 0] = {linear.weight.data[:, 0].tolist()}')
print(f'W[:, 2] = {linear.weight.data[:, 2].tolist()}')
print(f'W[:, 0] + W[:, 2] = {(linear.weight.data[:, 0] + linear.weight.data[:, 2]).tolist()}')
print(f'验证一致: {torch.allclose(y_spike, linear.weight.data[:, 0] + linear.weight.data[:, 2])}')

In [ ]:
# 测试3：不同spike模式 → 不同输出
spike_patterns = [
    [0, 0, 0, 0],  # 无spike
    [1, 0, 0, 0],  # 只有神经元0
    [0, 1, 0, 0],  # 只有神经元1
    [1, 1, 0, 0],  # 神经元0和1
    [1, 1, 1, 1],  # 全部发放
]

print('=== 不同spike模式 → 不同突触电流 ===')
print(f'{"Spike模式":20s} | {"输出(突触电流)":40s} | 含义')
print('=' * 90)
for sp in spike_patterns:
    x = torch.tensor(sp, dtype=torch.float32)
    y = linear(x)
    fired = [i for i, s in enumerate(sp) if s == 1]
    meaning = f'W的第{fired}列之和' if fired else '零电流'
    print(f'{str(sp):20s} | {str([round(v, 3) for v in y.tolist()]):40s} | {meaning}')

## 2. Linear + PLIF 组合：完整 SNN 层

In [ ]:
# 构建 Linear + PLIF 组合
snn_linear = layer.Linear(4, 3, bias=False, step_mode='s')
snn_neuron = neuron.ParametricLIFNode(
    init_tau=2.0, v_threshold=1.0, v_reset=None,  # v_reset=None → soft reset
    surrogate_function=surrogate.Sigmoid(),
    step_mode='s'
)

# 设置相同的权重
with torch.no_grad():
    snn_linear.weight.copy_(torch.tensor([
        [1.0, 0.5, -0.3, 0.2],
        [0.1, 0.8, 0.4, -0.5],
        [-0.2, 0.3, 0.9, 0.1]
    ]))

print('=== Linear + PLIF (soft reset) ===')
print(f'Linear: 4→3')
print(f'PLIF: tau={2.0}, V_th={1.0}, soft reset')
print(f'W = {snn_linear.weight.data.tolist()}')

In [ ]:
# 连续送入spike序列，观察完整流程
T = 20
torch.manual_seed(42)
# 随机spike输入 (4维, 每步有些发放有些不发放)
x_seq = (torch.rand(T, 4) > 0.5).float()

snn_neuron.reset()

currents = []  # Linear输出 = 突触电流
voltages = []  # PLIF膜电位
spikes = []    # PLIF输出spike

for t in range(T):
    # Step 1: Linear 把spike转成突触电流
    I_t = snn_linear(x_seq[t])
    currents.append(I_t.detach().clone())
    
    # Step 2: PLIF 接收电流，更新膜电位，判定是否发放
    s_t = snn_neuron(I_t)
    voltages.append(snn_neuron.v.detach().clone())
    spikes.append(s_t.detach().clone())

currents = torch.stack(currents).numpy()
voltages = torch.stack(voltages).numpy()
spikes = torch.stack(spikes).numpy()

print(f'输入: spike序列 {x_seq.shape} (每步4维, 0/1)')
print(f'Linear输出: 突触电流 {currents.shape} (每步3维, 实数)')
print(f'PLIF输出: spike序列 {spikes.shape} (每步3维, 0/1)')
print(f'\n输入唯一值: {np.unique(x_seq.numpy())}')
print(f'Linear输出值域: [{currents.min():.3f}, {currents.max():.3f}] (实数)')
print(f'PLIF输出唯一值: {np.unique(spikes)}')

In [ ]:
# 可视化完整流程：spike输入 → 突触电流 → 膜电位 → spike输出
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
t_arr = np.arange(T)

# 1. 输入spike
ax = axes[0]
for i in range(4):
    spike_times = np.where(x_seq[:, i].numpy() > 0.5)[0]
    ax.scatter(spike_times, np.full_like(spike_times, i), marker='|', s=100, linewidths=2)
ax.set_yticks(range(4))
ax.set_yticklabels([f'in_{i}' for i in range(4)])
ax.set_title('输入: spike (0/1) — 4维', fontweight='bold', loc='left')

# 2. Linear输出 = 突触电流
ax = axes[1]
for i in range(3):
    ax.plot(t_arr, currents[:, i], '-o', markersize=3, label=f'I_{i}')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
ax.set_title('Linear输出: 突触电流 I[t] = W @ spike[t] (实数) — W的作用', fontweight='bold', loc='left')
ax.legend(loc='upper right', fontsize=9)
ax.set_ylabel('电流')

# 3. PLIF膜电位
ax = axes[2]
for i in range(3):
    ax.plot(t_arr, voltages[:, i], '-', linewidth=1, label=f'V_{i}')
ax.axhline(y=1.0, color='red', linestyle='--', alpha=0.4, label='V_th=1.0')
ax.set_title('PLIF膜电位: V[t] = β·V[t-1] + I[t] (实数,内部状态)', fontweight='bold', loc='left')
ax.legend(loc='upper right', fontsize=9)
ax.set_ylabel('膜电位')

# 4. 输出spike
ax = axes[3]
for i in range(3):
    spike_times = np.where(spikes[:, i] > 0.5)[0]
    ax.scatter(spike_times, np.full_like(spike_times, i), marker='|', s=100, linewidths=2)
ax.set_yticks(range(3))
ax.set_yticklabels([f'out_{i}' for i in range(3)])
ax.set_title('PLIF输出: spike (0/1) — 3维', fontweight='bold', loc='left')
ax.set_xlabel('Time step')

fig.suptitle('完整 SNN 层: spike → Linear(W) → 突触电流 → PLIF → spike', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print('\n总结:')
print('  输入: spike (0/1)')
print('  Linear(W): spike → 突触电流 (W选择发放神经元对应列求和)')
print('  PLIF: 突触电流 → 膜电位积累 → 阈值判定 → spike (0/1)')
print('  输出: spike (0/1)')

## 3. W 的作用：突触权重如何影响信息传递

In [ ]:
# 对比不同W对相同输入的影响
print('=== W 的物理意义 ===')
print()

# 相同输入: 神经元0发放
x = torch.tensor([1.0, 0.0, 0.0, 0.0])
print(f'输入: {x.tolist()} (只有神经元0发放)')
print()

# W的第0列决定了神经元0的spike对下游3个神经元注入多少电流
print(f'W[:, 0] = {snn_linear.weight.data[:, 0].tolist()}')
print(f'  → 神经元0发放时:')
print(f'    对下游神经元0注入电流: {snn_linear.weight.data[0, 0]:.1f} (兴奋性)')
print(f'    对下游神经元1注入电流: {snn_linear.weight.data[1, 0]:.1f} (弱兴奋性)')
print(f'    对下游神经元2注入电流: {snn_linear.weight.data[2, 0]:.1f} (抑制性)')
print()
print('W[j, i] > 0: 突触i→j是兴奋性的 (spike促进下游积累)')
print('W[j, i] < 0: 突触i→j是抑制性的 (spike抑制下游积累)')
print('W[j, i] ≈ 0: 突触i→j几乎无连接')
print()
print('W的每一列 = 一个前突触神经元发放时对所有后突触神经元的影响')
print('W的每一行 = 一个后突触神经元接收所有前突触神经元的影响')

In [ ]:
# 实数输入 vs spike输入通过Linear的区别
print('=== 实数输入 vs Spike输入 通过 Linear ===')
print()

x_real = torch.tensor([0.73, 0.21, 0.95, 0.44])
x_spike = torch.tensor([1.0, 0.0, 1.0, 0.0])

y_real = snn_linear(x_real)
y_spike = snn_linear(x_spike)

print(f'实数输入:  {x_real.tolist()}')
print(f'Linear输出: {[round(v, 4) for v in y_real.tolist()]}')
print(f'  → W各列的加权和, 权重=输入值 (连续加权)')
print()
print(f'Spike输入: {x_spike.tolist()}')
print(f'Linear输出: {[round(v, 4) for v in y_spike.tolist()]}')
print(f'  → W中发放列的直接求和 (选择性求和)')
print()
print('关键区别:')
print('  实数输入: 每个输入维度贡献连续权重, 输出是所有列的加权混合')
print('  Spike输入: 只有发放的维度贡献, 输出是被选中列的纯净求和')
print('  Spike的稀疏性 → 天然的列选择机制')

## 4. 汇总：数据流类型链

In [ ]:
print('''
╔══════════════════════════════════════════════════════════════════════╗
║              SpikingJelly SNN 层的数据流                            ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                     ║
║  [spike 0/1]  ──→  Linear(W)  ──→  [实数: 突触电流]                  ║
║                      │                                              ║
║                 W @ spike                                           ║
║                 = 发放列求和                                         ║
║                 = 突触电流注入                                       ║
║                                                                     ║
║  [实数: 突触电流]  ──→  PLIF  ──→  [spike 0/1]                       ║
║                         │                                           ║
║                  V[t] = β·V[t-1] + I[t]                             ║
║                  if V > V_th: spike, V -= V_th                      ║
║                                                                     ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                     ║
║  W 的角色:                                                          ║
║    · W[j,i] = 突触 i→j 的连接强度                                   ║
║    · W[j,i] > 0: 兴奋性突触                                         ║
║    · W[j,i] < 0: 抑制性突触                                         ║
║    · spike输入时: W做列选择 (哪些前突触发放→选哪些列)                  ║
║    · 实数输入时: W做加权求和 (标准矩阵乘法)                           ║
║                                                                     ║
║  完整一层 = Linear(W) + PLIF                                        ║
║    输入: spike (0/1) 或 实数                                         ║
║    中间: 实数 (突触电流)                                             ║
║    输出: spike (0/1)                                                 ║
║                                                                     ║
╚══════════════════════════════════════════════════════════════════════╝
''')